# Graph neat examples in Raku

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 4</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
November 2024

-----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).   
- Graph plotting with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [1]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Nested;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;

use JavaScript::D3;
use WWW::MermaidInk;

use paths;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => #1F1F1F, edge-thickness => 3, title-color => Ivory, vertex-size => 6}

------

## Array of SVG plots

- Elegant iteration using graph classes *partial names* 
- It is possible to make a row of SVG plots (made, say, by Graphviz DOT) using the HTML output magic cell: 

In [5]:
#%html
my %opts = node-width => 0.2, node-fill-color => 'SlateBlue', node-height => 0.2, node-font-size => 6, engine => 'neato', size => (3, 3), :svg;

<Cycle Star Wheel>
    .map({ Graph::{$_}.new(8).dot(|%opts) })
    .join("\n\n")


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 1--8 -->
 
 1--8 
 
 
<!-- 1--2 -->
 
 1--2

------

## Grid with holes

In this section we show how to subtract random neighborhoods from a grid graph and plot the result.

In [6]:
my $g = Graph::TriangularGrid.new(20, 40);
#my $g = Graph::Grid.new(25, 50);

#my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(60), d => 1).vertex-list;
#my $g2 = $g.subgraph($vset.keys)
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(60), d => 1) );

Graph(vertexes => 871, edges => 1813, directed => False)

In [7]:
my $g3 = $g2.neighborhood-graph($g2.vertex-list.pick(60), d => 2);
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;
my @gs = @comps.map({ $g3.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

10

In [8]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => $g2.vertex-coordinates,
        highlight => {Ivory => @highlight},
        :$background, 
        title-color => 'none', 
        width => 1400, 
        height => 600,
        edge-thickness => 4,
        edge-color => 'Gray',
        vertex-size => 0,
        vertex-color => 'Gray',
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )

Graphviz DOT rendering:

In [9]:
#%html
$g2.dot( 
    highlight => {Ivory => @highlight},
    :!node-labels, 
    node-width => 0,
    node-fill-color => 'Gray', 
    node-shape => 'point',
    graph-size => (14, 24),
    edge-color => 'Gray',
    edge-width => 28,
    engine => 'neato', 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 404 -->
 
 404 
 
 
<!-- 415 -->
 
 415 
 
 
<!-- 404--415 -->
 
 404--415 
 
 
<!-- 425 -->
 
 425 
 
 
<!-- 404--425 -->
 
 404--425 
 
 
<!-- 650 -->
 
 650 
 
 
<!-- 661 -->
 
 661 
 
 
<!-- 650--661 -->
 
 650--661 
 
 
<!-- 660 -->
 
 660 
 
 
<!-- 650--660 -->
 
 650--660 
 
 
<!-- 671 -->
 
 671 
 
 
<!-- 650--671 -->
 
 650--671 
 
 
<!-- 365 -->
 
 365 
 
 
<!-- 386 -->
 
 386 
 
 
<!-- 365--386 -->
 
 365--386 
 
 
<!-- 376 -->
 
 376 
 
 
<!-- 365--376 -->
 
 365--376 
 
 
<!-- 756 -->
 
 756 
 
 
<!-- 776 -->
 
 776 
 
 
<!-- 756--776 -->
 
 756--776 
 
 
<!-- 766 -->
 
 766 
 
 
<!-- 756--766 -->
 
 756--766 
 
 
<!-- 767 -->
 
 767 
 
 
<!-- 756--767 -->
 
 756--767 
 
 
<!-- 787 -->
 
 787 
 
 
<!-- 795 -->
 
 795 
 
 
<!-- 787--795 -->
 
 787--795 
 
 
<!-- 804 -->
 
 804 
 
 
<!-- 787--804 -->
 
 787--804 
 
 
<!-- 835 -->
 
 835 
 
 
<!-- 841 -->
 
 841 
 
 
<!-- 835--841 -->
 
 835--841 
 
 
<!-- 385 -->
 
 385 
 
 
<!-- 395 -->
 
 395 
 
 
<!-- 385--395 -->
 
 385--395 
 
 
<!-- 406 -->
 
 406 
 
 
<!-- 385--406 -->
 
 385--406 
 
 
<!-- 396 -->
 
 396 
 
 
<!-- 385--396 -->
 
 385--396 
 
 
<!-- 278 -->
 
 278 
 
 
<!-- 289 -->
 
 289 
 
 
<!-- 278--289 -->
 
 278--289 
 
 
<!-- 299 -->
 
 299 
 
 
<!-- 278--299 -->
 
 278--299 
 
 
<!-- 337 -->
 
 337 
 
 
<!-- 347 -->
 
 347 
 
 
<!-- 337--347 -->
 
 337--347 
 
 
<!-- 348 -->
 
 348 
 
 
<!-- 337--348 -->
 
 337--348 
 
 
<!-- 358 -->
 
 358 
 
 
<!-- 337--358 -->
 
 337--358 
 
 
<!-- 87 -->
 
 87 
 
 
<!-- 642 -->
 
 642 
 
 
<!-- 653 -->
 
 653 
 
 
<!-- 642--653 -->
 
 642--653 
 
 
<!-- 760 -->
 
 760 
 
 
<!-- 780 -->
 
 780 
 
 
<!-- 760--780 -->
 
 760--780 
 
 
<!-- 770 -->
 
 770 
 
 
<!-- 760--770 -->
 
 760--770 
 
 
<!-- 352 -->
 
 352 
 
 
<!-- 373 -->
 
 373 
 
 
<!-- 352--373 -->
 
 352--373 
 
 
<!-- 362 -->
 
 362 
 
 
<!-- 352--362 -->
 
 352--362 
 
 
<!-- 363 -->
 
 363 
 
 
<!-- 352--363 -->
 
 352--363 
 
 
<!-- 157 -->
 
 157 
 
 
<!-- 168 -->
 
 168 
 
 
<!-- 157--168 -->
 
 157--168 
 
 
<!-- 178 -->
 
 178 
 
 
<!-- 157--178 -->
 
 157--178 
 
 
<!-- 167 -->
 
 167 
 
 
<!-- 157--167 -->
 
 157--167 
 
 
<!-- 862 -->
 
 862 
 
 
<!-- 865 -->
 
 865 
 
 
<!-- 862--865 -->
 
 862--865 
 
 
<!-- 368 -->
 
 368 
 
 
<!-- 347--368 -->
 
 347--368 
 
 
<!-- 347--358 -->
 
 347--358 
 
 
<!-- 357 -->
 
 357 
 
 
<!-- 347--357 -->
 
 347--357 
 
 
<!-- 395--406 -->
 
 395--406 
 
 
<!-- 416 -->
 
 416 
 
 
<!-- 395--416 -->
 
 395--416 
 
 
<!-- 691 -->
 
 691 
 
 
<!-- 701 -->
 
 701 
 
 
<!-- 691--701 -->
 
 691--701 
 
 
<!-- 712 -->
 
 712 
 
 
<!-- 691--712 -->
 
 691--712 
 
 
<!-- 702 -->
 
 702 
 
 
<!-- 691--702 -->
 
 691--702 
 
 
<!-- 439 -->
 
 439 
 
 
<!-- 460 -->
 
 460 
 
 
<!-- 439--460 -->
 
 439--460 
 
 
<!-- 450 -->
 
 450 
 
 
<!-- 439--450 -->
 
 439--450 
 
 
<!-- 449 -->
 
 449 
 
 
<!-- 439--449 -->
 
 439--449 
 
 
<!-- 733 -->
 
 733 
 
 
<!-- 754 -->
 
 754 
 
 
<!-- 733--754 -->
 
 733--754 
 
 
<!-- 744 -->
 
 744 
 
 
<!-- 733--744 -->
 
 733--744 
 
 
<!-- 743 -->
 
 743 
 
 
<!-- 733--743 -->
 
 733--743 
 
 
<!-- 422 -->
 
 422 
 
 
<!-- 433 -->
 
 433 
 
 
<!-- 422--433 -->
 
 422--433 
 
 
<!-- 443 -->
 
 443 
 
 
<!-- 422--443 -->
 
 422--443 
 
 
<!-- 432 -->
 
 432 
 
 
<!-- 422--432 -->
 
 422--432 
 
 
<!-- 95 -->
 
 95 
 
 
<!-- 607 -->
 
 607 
 
 
<!-- 617 -->
 
 617 
 
 
<!-- 607--617 -->
 
 607--617 
 
 
<!-- 628 -->
 
 628 
 
 
<!-- 607--628 -->
 
 607--628 
 
 
<!-- 34 -->
 
 34 
 
 
<!-- 39 -->
 
 39 
 
 
<!-- 34--39 -->
 
 34--39 
 
 
<!-- 46 -->
 
 46 
 
 
<!-- 34--46 -->
 
 34--46 
 
 
<!-- 40 -->
 
 40 
 
 
<!-- 34--40 -->
 
 34--40 
 
 
<!-- 211 -->
 
 211 
 
 
<!-- 221 -->
 
 221 
 
 
<!-- 211--221 -->
 
 211--221 
 
 
<!-- 48 -->
 
 48

-----

## Subgraph using vertex coordinates

Same as above but with ***Hexagonal Grid***:

In [10]:
my $g = Graph::HexagonalGrid.new(12, 24);
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(60), d => 1) );

Graph(vertexes => 650, edges => 744, directed => False)

Summary of the coordinates:

In [11]:
sink records-summary($g2.vertex-coordinates.values, field-names => ['0', '1'])

+-----------------------------+------------------------------+
| 0                           | 1                            |
+-----------------------------+------------------------------+
| Min    => -5.196152422707   | Min    => -4                 |
| 1st-Qu => 24.248711305964   | 1st-Qu => 5                  |
| Mean   => 46.04057515873089 | Mean   => 14.541538461538462 |
| Median => 46.76537180436    | Median => 15                 |
| 3rd-Qu => 67.549981495186   | 3rd-Qu => 24                 |
| Max    => 96.994845223857   | Max    => 33                 |
+-----------------------------+------------------------------+


Derive rectangular area boundaries:

In [12]:
my ($min-y, $max-y) = $g2.vertex-coordinates.values.map(*.tail).Array.&{ (.min, .max)};

my $new-min-x = $g2.vertex-coordinates.values.grep(*.tail == $max-y).map(*.head).min;
my $new-max-x = $g2.vertex-coordinates.values.grep(*.tail == $min-y).map(*.head).max;

76.210235533031

Mean point of graph's vertexes:

In [13]:

my @mean-point = |$g2.vertex-coordinates.values.Array.&transpose.map({ $_.sum/$_.elems });

[46.04057515873089 14.541538461538462]

Extract subgraph:

In [14]:
#my $g3 = $g2.subgraph($g2.vertex-coordinates.grep($new-min-x≤*.value.head≤$new-max-x)».key)

()

In [15]:
my @gs = 
[&euclidean-distance, &manhattan-distance, &chessboard-distance]
.map(-> &func { $g2.subgraph($g2.vertex-coordinates.grep({ &func($_.value, @mean-point) ≤ 16 })».key) })

[Graph(vertexes => 160, edges => 194, directed => False) Graph(vertexes => 111, edges => 131, directed => False) Graph(vertexes => 203, edges => 247, directed => False)]

DOT-plot (highlight vertexes with degree ≤ 2):

In [16]:
#%html
@gs.map( -> $g { $g.dot( 
    highlight => {IndianRed => $g.vertex-degree(:p).grep(*.value≤2)».key},
    :!node-labels, 
    node-width => 1.2,
    node-height => 1.2,
    node-fill-color => '#FFC000', 
    node-shape => 'circle',
    graph-size => (4, 4),
    edge-color => 'Tan',
    edge-width => 28,
    engine => 'neato', 
):svg }).join("\n\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 262 -->
 
 262 
 
 
<!-- 275 -->
 
 275 
 
 
<!-- 262--275 -->
 
 262--275 
 
 
<!-- 263 -->
 
 263 
 
 
<!-- 262--263 -->
 
 262--263 
 
 
<!-- 238 -->
 
 238 
 
 
<!-- 251 -->
 
 251 
 
 
<!-- 238--251 -->
 
 238--251 
 
 
<!-- 239 -->
 
 239 
 
 
<!-- 238--239 -->
 
 238--239 
 
 
<!-- 308 -->
 
 308 
 
 
<!-- 309 -->
 
 309 
 
 
<!-- 308--309 -->
 
 308--309 
 
 
<!-- 321 -->
 
 321 
 
 
<!-- 308--321 -->
 
 308--321 
 
 
<!-- 380 -->
 
 380 
 
 
<!-- 393 -->
 
 393 
 
 
<!-- 380--393 -->
 
 380--393 
 
 
<!-- 381 -->
 
 381 
 
 
<!-- 380--381 -->
 
 380--381 
 
 
<!-- 445 -->
 
 445 
 
 
<!-- 299 -->
 
 299 
 
 
<!-- 312 -->
 
 312 
 
 
<!-- 299--312 -->
 
 299--312 
 
 
<!-- 292 -->
 
 292 
 
 
<!-- 293 -->
 
 293 
 
 
<!-- 292--293 -->
 
 292--293 
 
 
<!-- 305 -->
 
 305 
 
 
<!-- 292--305 -->
 
 292--305 
 
 
<!-- 370 -->
 
 370 
 
 
<!-- 371 -->
 
 371 
 
 
<!-- 370--371 -->
 
 370--371 
 
 
<!-- 383 -->
 
 383 
 
 
<!-- 370--383 -->
 
 370--383 
 
 
<!-- 344 -->
 
 344 
 
 
<!-- 345 -->
 
 345 
 
 
<!-- 344--345 -->
 
 344--345 
 
 
<!-- 357 -->
 
 357 
 
 
<!-- 344--357 -->
 
 344--357 
 
 
<!-- 394 -->
 
 394 
 
 
<!-- 407 -->
 
 407 
 
 
<!-- 394--407 -->
 
 394--407 
 
 
<!-- 395 -->
 
 395 
 
 
<!-- 394--395 -->
 
 394--395 
 
 
<!-- 410 -->
 
 410 
 
 
<!-- 423 -->
 
 423 
 
 
<!-- 410--423 -->
 
 410--423 
 
 
<!-- 411 -->
 
 411 
 
 
<!-- 410--411 -->
 
 410--411 
 
 
<!-- 290 -->
 
 290 
 
 
<!-- 291 -->
 
 291 
 
 
<!-- 290--291 -->
 
 290--291 
 
 
<!-- 303 -->
 
 303 
 
 
<!-- 290--303 -->
 
 290--303 
 
 
<!-- 384 -->
 
 384 
 
 
<!-- 371--384 -->
 
 371--384 
 
 
<!-- 420 -->
 
 420 
 
 
<!-- 407--420 -->
 
 407--420 
 
 
<!-- 404 -->
 
 404 
 
 
<!-- 405 -->
 
 405 
 
 
<!-- 404--405 -->
 
 404--405 
 
 
<!-- 288 -->
 
 288 
 
 
<!-- 275--288 -->
 
 275--288 
 
 
<!-- 255 -->
 
 255 
 
 
<!-- 268 -->
 
 268 
 
 
<!-- 255--268 -->
 
 255--268 
 
 
<!-- 277 -->
 
 277 
 
 
<!-- 277--290 -->
 
 277--290 
 
 
<!-- 418 -->
 
 418 
 
 
<!-- 419 -->
 
 419 
 
 
<!-- 418--419 -->
 
 418--419 
 
 
<!-- 316 -->
 
 316 
 
 
<!-- 317 -->
 
 317 
 
 
<!-- 316--317 -->
 
 316--317 
 
 
<!-- 329 -->
 
 329 
 
 
<!-- 316--329 -->
 
 316--329 
 
 
<!-- 254 -->
 
 254 
 
 
<!-- 254--255 -->
 
 254--255 
 
 
<!-- 267 -->
 
 267 
 
 
<!-- 254--267 -->
 
 254--267 
 
 
<!-- 343 -->
 
 343 
 
 
<!-- 356 -->
 
 356 
 
 
<!-- 343--356 -->
 
 343--356 
 
 
<!-- 304 -->
 
 304 
 
 
<!-- 291--304 -->
 
 291--304 
 
 
<!-- 276 -->
 
 276 
 
 
<!-- 276--277 -->
 
 276--277 
 
 
<!-- 289 -->
 
 289 
 
 
<!-- 276--289 -->
 
 276--289 
 
 
<!-- 396 -->
 
 396 
 
 
<!-- 383--396 -->
 
 383--396 
 
 
<!-- 256 -->
 
 256 
 
 
<!-- 269 -->
 
 269 
 
 
<!-- 256--269 -->
 
 256--269 
 
 
<!-- 257 -->
 
 257 
 
 
<!-- 256--257 -->
 
 256--257 
 
 
<!-- 436 -->
 
 436 
 
 
<!-- 423--436 -->
 
 423--436 
 
 
<!-- 241 -->
 
 241 
 
 
<!-- 241--254 -->
 
 241--254 
 
 
<!-- 212 -->
 
 212 
 
 
<!-- 213 -->
 
 213 
 
 
<!-- 212--213 -->
 
 212--213 
 
 
<!-- 225 -->
 
 225 
 
 
<!-- 212--225 -->
 
 212--225 
 
 
<!-- 406 -->
 
 406 
 
 
<!-- 393--406 -->
 
 393--406 
 
 
<!-- 382 -->
 
 382 
 
 
<!-- 382--383 -->
 
 382--383 
 
 
<!-- 382--395 -->
 
 382--395 
 
 
<!-- 322 -->
 
 322 
 
 
<!-- 335 -->
 
 335 
 
 
<!-- 322--335 -->
 
 322--335 
 
 
<!-- 323 -->
 
 323 
 
 
<!-- 322--323 -->
 
 322--323 
 
 
<!-- 339 -->
 
 339 
 
 
<!-- 352 -->
 
 352 
 
 
<!-- 339--352 -->
 
 339--352 
 
 
<!-- 243 -->
 
 243 
 
 
<!-- 243--256 -->
 
 243--256 
 
 
<!-- 408 -->
 
 408 
 
 
<!-- 395--408 -->
 
 395--408 
 
 
<!-- 236 -->
 
 236 
 
 
<!-- 249 -->
 
 249 
 
 
<!-- 236--249 -->
 
 236--249 
 
 
<!-- 237 -->
 
 237 
 
 
<!-- 236--237 -->
 
 236--237 
 
 
<!-- 353 -->
 
 353 
 
 
<!-- 366 -->
 
 366 
 
 
<!-